# 自然言語処理

## MeCabの準備

***

### 導入手順（Windows 10）

1. [MeCab](https://github.com/ikegami-yukino/mecab/releases/tag/v0.996)をダウンロード・インストールする。
<br />※公式の32bit版ではなく、有志がビルドした64bit版を使用する。
1. システムのプロパティから環境変数の編集で"Path"を選択し、以下のパスを追加する。
 * `（MeCabインストール先）\bin`
1. （仮想環境上の）コマンドプロンプトを起動し、以下のコマンドを実行し、インストールする。
 * `pip install ipykernel` \# 最初からAnaconda環境に含まれている
 * `pip install mecab-python-windows`
1. インストールが完了したら、`（MeCabインストールパス）\bin`にあるファイル`libmecab.dll`を以下のフォルダにコピー & ペーストする。
 * `（Anacondaインストール先）\Lib\site-packages`
1. 以上でpythonからMeCabが実行可能になる。

### 利用方法

```python
import MeCab
import re

# テキストの設定
text = 'すもももももももものうち。'

# 形態素解析
header = '表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音'
parse = MeCab.Tagger().parse(text)

lines = list()
lines.append(header)
lines.extend(parse.split('\n'))
items = [re.split('[\t,]', line) for line in lines if len(re.split('[\t,]', line)) == 10]

# 結果を表示
for item in items:
    print(item)
```

### 参考資料

* [PythonとMeCabで形態素解析(on Windows) - Qiita](https://qiita.com/menon/items/f041b7c46543f38f78f7)
* [PythonでMeCabを使ってみる(Windows10 64bit) - Qiita](https://qiita.com/wanko5296/items/eeb7865ee71a7b9f1a3a)

## ユーザ辞書（csv）の作成・コンパイル

***

In [2]:
import pandas

def MeCabDictionary(MeCabDictionaryList):
    '''
    outline : 
    input : list([string, string, string, string, string, string], )
    output : pandas.dataframe
    '''
    columns = ['表層形', '左文脈ID', '右文脈ID', 'コスト', '品詞', '品詞細分類1', '品詞細分類2', '品詞細分類3', '活用形', '活用型', '原形', '読み', '発音']
    data = list()
    for item in MeCabDictionaryList:
        data.append([item[0], 1, 1, 1, item[1], item[2], '*', '*', '*', '*', item[3], item[4], item[5]])
    return pandas.DataFrame(data=data, columns=columns)
    
registerWords = [
    ['言葉', '名詞', '一般', '言葉', 'コトバ', 'コトバ'],
    ['すもももももももものうち', '名詞', '一般', 'すもももももももものうち', 'スモモモモモモモモノウチ', 'スモモモモモモモモノウチ']
]

fileName = 'user_mecab'
csvDataFrame = MeCabDictionary(registerWords)
csvDataFrame.to_csv('csv\{0}.csv'.format(fileName), header=False, index=False)

print('csvファイルを作成しました。csvファイル名: {0}'.format(fileName)) 
csvDataFrame

csvファイルを作成しました。csvファイル名: user_mecab


,表層形,左文脈ID,右文脈ID,コスト,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
0,言葉,1,1,1,名詞,一般,*,*,*,*,言葉,コトバ,コトバ
1,すもももももももものうち,1,1,1,名詞,一般,*,*,*,*,すもももももももものうち,スモモモモモモモモノウチ,スモモモモモモモモノウチ


In [1]:
import subprocess

mecabPath = r'c:\Program Files\MeCab'
currentPath = r'c:\Users\hajime\ipynb'
fileName = 'user_mecab'
commands = list()
commands.append(r'"{0}\bin\mecab-dict-index.exe" -d "{0}\dic\ipadic" -u {1}.dic -f utf-8 -t utf-8 {1}.csv'.format(mecabPath, fileName))
commands.append(r'move {1}.dic {0}\dic'.format(currentPath, fileName))
for command in commands:
    proc = subprocess.run(command, cwd=r'{0}\csv'.format(currentPath), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=3)
    print(proc.stdout.decode('cp932'))
    
print('ユーザ辞書を作成しました。')

reading user_mecab.csv ... 2
emitting double-array: 100% |###########################################| 

done!

        1 個のファイルを移動しました。

ユーザ辞書を作成しました。


## ユーザ辞書の指定

***

In [6]:
import MeCab
import re

def ParsesWithUserDic(text):
    '''
    outline: 
    
    @param: 
    @type: 
    @rtype: 
    '''
    userDic = 'user_mecab'
    
    # 形態素解析
    header = '表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音'
    parse = MeCab.Tagger('-u dic/{0}.dic'.format(fileName)).parse(text)
    
    lines = list()
    lines.append(header)
    lines.extend(parse.split('\n'))
    items = [re.split('[\t,]', line) for line in lines if len(re.split('[\t,]', line)) == 10]
    
    # 結果を出力する
    return items

print(ParsesWithUserDic('すもももももももものうち'))

[['表層形', '品詞', '品詞細分類1', '品詞細分類2', '品詞細分類3', '活用型', '活用形', '原形', '読み', '発音'], ['すもももももももものうち', '名詞', '一般', '*', '*', '*', '*', 'すもももももももものうち', 'スモモモモモモモモノウチ', 'スモモモモモモモモノウチ']]


## MeCabの実行

***

In [7]:
import MeCab
import re

def Parses(text):
    '''
    outline: 形態素データを取得する
    
    @param text: C(str)
    @type text: C(list)
    @rtype: C(list)
    '''
    # 形態素解析
    header = '表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音'
    parse = MeCab.Tagger().parse(text)
    
    lines = list()
    lines.append(header)
    lines.extend(parse.split('\n'))
    items = [re.split('[\t,]', line) for line in lines if len(re.split('[\t,]', line)) == 10]
    
    # 結果を出力する
    return items

help(Parses)
print(Parses('すもももももももものうち'))

Help on function Parses in module __main__:

Parses(text)
    outline: 形態素データを取得する
    
    @param text: C(str)
    @type text: C(list)
    @rtype: C(list)

[['表層形', '品詞', '品詞細分類1', '品詞細分類2', '品詞細分類3', '活用型', '活用形', '原形', '読み', '発音'], ['すもも', '名詞', '一般', '*', '*', '*', '*', 'すもも', 'スモモ', 'スモモ'], ['も', '助詞', '係助詞', '*', '*', '*', '*', 'も', 'モ', 'モ'], ['もも', '名詞', '一般', '*', '*', '*', '*', 'もも', 'モモ', 'モモ'], ['も', '助詞', '係助詞', '*', '*', '*', '*', 'も', 'モ', 'モ'], ['もも', '名詞', '一般', '*', '*', '*', '*', 'もも', 'モモ', 'モモ'], ['の', '助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 'ノ'], ['うち', '名詞', '非自立', '副詞可能', '*', '*', '*', 'うち', 'ウチ', 'ウチ']]


## 文の分割

***

In [ ]:
def GetParsesPart(text, index):
    '''
    outline : 
    input : text, index
    output : list(string)
    '''
    parts = [wordParse[index] for wordParse in Parses(text)]
    del parts[0] # header情報の削除
    return parts

def SplitWord(text):
    '''
    outline : 
    input : text
    output : list(string)
    '''
    return GetParsesPart(text, 0)

def SplitClass(text):
    '''
    outline : 
    input : text
    output : list(string)
    '''
    return GetParsesPart(text, 1)
    
print(', '.join(SplitWord('すもももももももものうち。')))
print(', '.join(SplitClass('すもももももももものうち。')))

## HTMLテキストの処理

***

In [ ]:
from urllib.request import urlopen
from urllib import parse
from bs4 import BeautifulSoup
import re

def HTMLText(url):
    '''
    input : text
    output : text
    outline : urlのwebページにアクセスし、webページのテキストデータを取得する
    '''
    html = urlopen(url)
    
    # 結果を出力する
    return BeautifulSoup(html.read(), 'html.parser')

def GetUrlText(hostName, parameter):
    '''
    '''
    return hostName + parse.quote(parameter)

In [ ]:
hostName = 'https://ja.wikipedia.org/wiki/'
targetNames = ['洗剤', 'ウィキペディア', '日本語', 'マスク', 'スプレー', '本', '会社', 'パソコン', '鞄', '生物']

for targetName in targetNames:
    bs = HTMLText(GetUrlText(hostName, targetName))
    text = re.sub(r'\n+', '\n', bs.find('div', {'class':'mw-parser-output'}).get_text())
    textFileName = 'txt\{0}.txt'.format(targetName)
    textFile = open(textFileName, mode='w', encoding='utf-8')
    
    for line in re.split(r'\n', text):
        textFile.write(line+'\n')
    textFile.close()
    
print('テキストファイルを作成しました。テキストファイル名: {0}'.format(', '.join(targetNames))) 

In [ ]:
# pairList = [[argWord, argClass] for (argWord, argClass) in zip(SplitWord(text), SplitClass(text))]
# print(pairList[:5])
# 
# classes = set()
# targets = set()
# 
# for item in [pair[1] for pair in pairList]:
#     classes.add(item)
# print(classes)
# 
# for item in [pair[0] for pair in pairList if pair[1] == '副詞']:
#     targets.add(item)
# print(targets)
# 
# for sentence in re.split(r'。', text):
#     print(sentence + '。')